In [4]:
import json
import pandas as pd
import numpy as np

In [5]:
file_dir = 'C://Users/fenwa/Desktop/Git/Movies-ETL/'

In [6]:
with open(f'{file_dir}wikipedia-movies.json', mode='r') as file:
    wiki_movies_raw = json.load(file)

In [7]:
len(wiki_movies_raw)

7311

In [8]:
#first 5 records
#wiki_movies_raw[:5]

In [9]:
# Last 5 records
#wiki_movies_raw[-5:]

In [10]:
# Some records in the middle
#wiki_movies_raw[3600:3605]

In [11]:
kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
ratings = pd.read_csv(f'{file_dir}ratings.csv')

In [12]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [13]:
#wiki_movies_df.head()

In [14]:
#wiki_movies_df.columns.tolist()

In [15]:
#[expression for element in source_list]
#[expression for element in source_list if filter_expression]
wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

In [16]:
wiki_movies_only_df = pd.DataFrame(wiki_movies)

In [17]:
#wiki_movies_only_df.head()

In [18]:
#wiki_movies_only_df.columns.tolist()

In [19]:
#we can remove no. episodes because it's only for tv. 

In [20]:
#moving this process to a function to make it easier
#Lambda = one line functions; lambda arguments: expression
#def + : makes a function
def clean_movie(movie):
    # make a copy of movie so it is non-destructive
    # the 'movie' below is a local variable and doenst mess with the global movie variable!
    movie = dict(movie)
    return movie

In [21]:
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]
wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

7060    https://en.wikipedia.org/wiki/The_Insult_(film)
7293     https://en.wikipedia.org/wiki/Capernaum_(film)
Name: url, dtype: object

In [22]:
#sorted(wiki_movies_df.columns.tolist())

In [23]:
## born/died/family/gender/parents/parent(s) is probably for people and not relevant to movies - maybe remove later?
## engines is probably for video games
#Also known as
#arabic
#bopomofo - Chinese
#Cantonese
#Chinese
#French
#Gwoyeu Romatzyh - more Chinese
#Hangul - Korean
#Hanyu Pinyin - Chinese
#Hebrew
#Hepburn - Japanese
#Hokkien POJ - Taiwanese Chinese
#IPA Chinese
#Japanese
#Jyutping - Chinese
#Mandarin
#McCune-Reischaur
#Polish
#Revised Romanization - Korean
#Romanized - unlclear which language is being Romanized
#Russian
#Simplified - Probably Chinese
#Simplified Chinese
#Traditional - Probably Chinese
#Traditional Chinese
#Wade-Giles - Chinese
#Yale Romanization - Chinese
#Yiddish


In [24]:
def clean_movie(movie):
    movie = dict(movie)
    alt_titles = {}
    for key in ['Also known as', 'Arabic', 'Bopomofo','Cantonese','Chinese','French',
                'Gwoyeu Romatzyh','Hangul','Hebrew','Hokkien POJ', 'IPA Chinese',
                'Japanese','Jyutping','Mandarin','McCune–Reischauer','Polish',
                'Revised Romanization','Romanized','Russian','Simplified',
                'Simplified Chinese', 'Traditional','Traditional Chinese',
                'Wade-Giles','Yale Romanization','Yiddish', 'Hepburn']:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
    return movie

In [25]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [26]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())

['Adaptation by',
 'Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Country of origin',
 'Created by',
 'Directed by',
 'Director',
 'Distributed by',
 'Distributor',
 'Edited by',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Length',
 'Literally',
 'Music by',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Original release',
 'Original title',
 'Picture format',
 'Preceded by',
 'Produced by',
 'Producer',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Productioncompanies ',
 'Productioncompany ',
 'Recorded',
 'Release date',
 'Released',
 'Running time',
 'Screen story by',
 'Screenplay by',
 'Starring',
 'Story by',
 'Suggested by',
 'Theme music composer',
 'Venue',
 'Voices of',
 'Written by',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']

In [27]:
wiki_movies_df.head()

,url,year,imdb_link,title,Directed by,Produced by,Screenplay by,Story by,Based on,Starring,...,Venue,Length,Label,Director,Producer,Animation by,Color process,Screen story by,Original title,Literally
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,Renny Harlin,"[Steve Perry, Joel Silver]","[David Arnott, James Cappe, Daniel Waters]","[David Arnott, James Cappe]","[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet",James Foley,"[Ric Kidney, Robert Redlin]","[James Foley, Robert Redlin]",NaN,"[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://en.wikipedia.org/wiki/Air_America_(film),1990,https://www.imdb.com/title/tt0099005/,Air America,Roger Spottiswoode,Daniel Melnick,"[John Eskow, Richard Rush]",NaN,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990,https://www.imdb.com/title/tt0099012/,Alice,Woody Allen,Robert Greenhut,NaN,NaN,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990,https://www.imdb.com/title/tt0099018/,Almost an Angel,John Cornell,John Cornell,NaN,NaN,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
wiki_movies_df['alt_titles'].notnull()

0       False
1       False
2       False
3       False
4       False
        ...  
7071    False
7072    False
7073    False
7074    False
7075    False
Name: alt_titles, Length: 7076, dtype: bool